# Stochastic Gradient Descent für MFCC

In [1]:
max_file_count = None
none_replacement = 'all'

RANDOM_STATE = 42

## MFCC Daten laden

🔗 siehe [generate.ipynb](./generate.ipynb)

In [2]:
import numpy as np

feature_matrix = np.load(f'feature_{max_file_count or none_replacement}.npy')
labels = np.load(f'labels_{max_file_count or none_replacement}.npy')

## Aufteilung in Trainings- und Testdaten

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.abs(feature_matrix), labels,
                                                    test_size=0.2, random_state=RANDOM_STATE)

In [4]:
print(X_train.shape)
print(X_test.shape)

(31697, 44)
(7925, 44)


## Training des SGD

In [5]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(n_jobs=4, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

SGDClassifier(n_jobs=4, random_state=42)

⚠️ `random_state` hat hohen Einfluss auf Leistung!

In [6]:
import skops.io as sio
sio.dump(model, f'sgd_{max_file_count or none_replacement}.skops')

## Leistung des Modells

In [7]:
y_test_predict = model.predict(X_test)

In [8]:
from sklearn import metrics

# proportion of correct predictions
print(f'accuracy:  {metrics.accuracy_score(y_test, y_test_predict):.4f}')

# proportion of true positives under all positives
print(f'precision: {metrics.precision_score(y_test, y_test_predict):.4f}')

# proportion of true positives under actually positives
print(f'recall:    {metrics.recall_score(y_test, y_test_predict):.4f}')

# total performance
print(f'f1:        {metrics.f1_score(y_test, y_test_predict):.4f}')

accuracy:  0.9869
precision: 0.0811
recall:    0.0411
f1:        0.0545


In [9]:
print(metrics.classification_report(y_test, y_test_predict, zero_division=0))

              precision    recall  f1-score   support

          -1       0.99      1.00      0.99      7852
           1       0.08      0.04      0.05        73

    accuracy                           0.99      7925
   macro avg       0.54      0.52      0.52      7925
weighted avg       0.98      0.99      0.98      7925



In [10]:
_ = metrics.ConfusionMatrixDisplay.from_predictions(y_test, y_test_predict)